In [12]:
# ideas for a source test
import cobra
import pandas as pd
from typing import Literal
import warnings

import refinegems as rg

In [13]:
test_model_path = '/Users/carolinb/Documents/104 Masterthesis/klebsiella-pipeline/example/thesis/Kp_std/03_refinement/step4-smoothing/Kp_std_smooth.xml'
test_model = rg.io.load_model_cobra(test_model_path)
e = 'N'
substances = None
medium = None
dir = '/Users/carolinb/Documents/200 Workspace/testing/rg/'

source_res = rg.growth.test_growth_with_source(test_model, e, substances, medium)

# source_res.save(dir)
